In [2]:
import pandas as pd
import datetime as dt
import numpy as np

SET DE DATOS

In [3]:
eventsData = pd.read_csv("data/events_up_to_01062018.csv",low_memory = False)

In [4]:
trainingSet = pd.read_csv("data/labels_training_set.csv", low_memory = False)

In [5]:
kaggle_test = pd.read_csv("data/trocafone_kaggle_test.csv", low_memory = False)

Funciones para agregar datos

In [6]:
def agregarNacionalidadRegionYCiudad(df):
    userscountry = {}
    usersregion ={}
    userscity = {}
    device_type = {}
    screen_resolution = {}
    operating_system_version = {}
    browser_version = {}
    new_vs_returning = {}
    channel={}
    df = eventsData
    df = df[["event","person","country","region","city","screen_resolution","device_type","operating_system_version","browser_version","new_vs_returning","channel"]]
    df= df[df["event"] == "visited site"]
    df = df.drop_duplicates(keep="first")
    df = df.set_index("person")
    for user, row in df.iterrows():
        userscountry[user] = row["country"]
        usersregion[user] =row["region"]
        userscity[user] = row["city"]
        device_type[user] = row["device_type"]
        screen_resolution[user]=row["screen_resolution"]
        operating_system_version[user] = row["operating_system_version"]
        browser_version[user] = row["browser_version"]
        new_vs_returning[user] = row["new_vs_returning"]
        channel[user] = row["channel"]
    return userscountry,usersregion,userscity,device_type,screen_resolution,operating_system_version,browser_version,new_vs_returning,channel

In [7]:
def agregarModelos(df):
    skuModels = {}
    
    df = df[["event","sku","model"]]
    df= df[df["event"] == "checkout"]
    df = df.drop_duplicates(keep="first")
    df = df.set_index("sku")
    for sku, row in df.iterrows():
        skuModels[sku] = row["model"]
    return skuModels

In [8]:
def capitalize_words(string):
    words = string.split()
    return ' '.join([word.capitalize() for word in words])




In [9]:
urls={}
for url in eventsData["url"].dropna():
    if(len(url)>1):
        stringModelo = url.split("/")[2:]
        if(len(stringModelo)>1):
            
            stringModelo1 = stringModelo[1].split("-")
            
            if(stringModelo[0] == stringModelo1[0]):
                urls[url] = capitalize_words(" ".join(stringModelo1))
            else:
                urls[url] = capitalize_words(" ".join([stringModelo[0]]+stringModelo1))
        else:
            #print(stringModelo)
            urls[url] = capitalize_words(" ".join(stringModelo))

In [10]:
#Funcion que pasa a formato datetime las fechas que recibimos.
def datetime(date,format_string):
    return pd.to_datetime(date,format= format_string,errors='coerce')




In [11]:
def DevolverParteDelDia(date):
    if(date.hour > 0 and date.hour<6):
        return "Madrugada"
    elif(date.hour > 6 and date.hour<12):
        return "Mañana"
    elif(date.hour>12 and date.hour<18):
        return "Tarde"
    else:
        return "Noche"

Agrego Nacionalidades faltantes

In [12]:
userscountry,usersregion,userscity,device_type,screen_resolution,operating_system_version,browser_version,new_vs_returning,channel = agregarNacionalidadRegionYCiudad(eventsData)
eventsData["country"] = eventsData["country"].fillna(eventsData["person"].map(userscountry))
eventsData["region"] = eventsData["region"].fillna(eventsData["person"].map(usersregion))
eventsData["city"] = eventsData["city"].fillna(eventsData["person"].map(userscity))
eventsData["device_type"] = eventsData["device_type"].fillna(eventsData["person"].map(device_type))
eventsData["screen_resolution"] = eventsData["screen_resolution"].fillna(eventsData["person"].map(screen_resolution))
eventsData["operating_system_version"] = eventsData["operating_system_version"].fillna(eventsData["person"].map(operating_system_version))
eventsData["browser_version"] = eventsData["browser_version"].fillna(eventsData["person"].map(browser_version))
#eventsData["new_vs_returning"] = eventsData["new_vs_returning"].fillna(eventsData["person"].map(new_vs_returning))
#eventsData["channel"] = eventsData["channel"].fillna(eventsData["person"].map(channel))

Agrego modelos faltantes

In [13]:
eventsData["model"].isnull().value_counts()

False    1321513
True     1020168
Name: model, dtype: int64

In [14]:
eventsData["model"] = eventsData["model"].fillna(eventsData["url"].map(urls))

In [15]:
eventsData["model"] = eventsData["model"].map(lambda x: x.lower() if ((not pd.isnull(x)) )else x)

In [16]:
eventsData["model"].isnull().value_counts()

False    1448457
True      893224
Name: model, dtype: int64

Agrego parte del dia 

In [17]:
eventsData["timestamp"] = datetime(eventsData["timestamp"],"%Y-%m-%dT%H:%M:%S")

In [18]:
eventsData["Parte del dia"] = eventsData["timestamp"].map(lambda x: DevolverParteDelDia(x))

In [19]:
eventsData.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,Parte del dia
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,samsung galaxy j7 prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,Cantagalo,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,Noche
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iphone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,Noche
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iphone 6s,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,Niterói,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 61.0,Noche
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iphone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,Juquitiba,Sao Paulo,Brazil,Computer,1360x768,Windows 8.1,Chrome 66.0,Noche
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,lg g4 h818p,Excelente,32GB,Branco,NaN,...,NaN,NaN,Osasco,Sao Paulo,Brazil,Computer,1536x864,Windows 7,Chrome 66.0,Noche


In [20]:
eventsData.to_csv('Eventos(+completo).csv')